Cet ensemble de données contient des informations sur les valeurs foncières géolocalisées du 01/01/2024 au 30/06/2024, avec des colonnes détaillant les transactions immobilières : identifiants de mutation, dates, valeurs foncières, adresses complètes (numéro, voie, code postal, commune), types de biens (surface, nombre de pièces), et localisation (latitude, longitude). Ces données permettent d'analyser les tendances foncières et immobilières sur le territoire français. Les données sont disponibles sur : https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/.

# Description des variables

Pour plus d'informations : https://doc-datafoncier.cerema.fr/doc/dv3f/mutation/nbdispo 

| Nom de la variable                 | Type         | Description                                                                                  |
|------------------------------------|--------------|----------------------------------------------------------------------------------------------|
| id_mutation                        | object       | Identifiant de mutation (non stable, sert à grouper les lignes)                              |
| date_mutation                      | object       | Date de la mutation au format ISO-8601 (YYYY-MM-DD)                                          |
| numero_disposition                 | int64        | Numéro de la disposition au sein d'une mutation. Une mutation peut avoir plusieurs dispositions |
| nature_mutation                    | object       | Nature de la mutation                                                                         |
| valeur_fonciere                    | float64      | Valeur foncière (séparateur décimal = point): chaque id_mutation correspond à une seule et unique valeur de valeur_fonciere |
| adresse_numero                     | float64      | Numéro de l'adresse                                                                            |
| adresse_suffixe                    | object       | Suffixe du numéro de l'adresse (B, T, Q)                                                      |
| adresse_code_voie                  | object       | Code FANTOIR de la voie (4 caractères)                                                        |
| adresse_nom_voie                   | object       | Nom de la voie de l'adresse                                                                   |
| code_postal                        | float64      | Code postal (5 caractères)                                                                   |
| code_commune                       | object       | Code commune INSEE (5 caractères)                                                            |
| nom_commune                        | object       | Nom de la commune (accentué)                                                                  |
| ancien_code_commune                | float64      | Ancien code commune INSEE (si différent lors de la mutation)                                 |
| ancien_nom_commune                 | object       | Ancien nom de la commune (si différent lors de la mutation)                                  |
| code_departement                   | object       | Code département INSEE (2 ou 3 caractères)                                                    |
| id_parcelle                        | object       | Identifiant de parcelle (14 caractères)                                                      |
| ancien_id_parcelle                 | float64      | Ancien identifiant de parcelle (si différent lors de la mutation)                            |
| numero_volume                      | object       | Numéro de volume                                                                              |
| lot_1_numero                       | object       | Numéro du lot 1                                                                               |
| lot_1_surface_carrez               | float64      | Surface Carrez du lot 1                                                                       |
| lot_2_numero                       | object       | Numéro du lot 2                                                                               |
| lot_2_surface_carrez               | float64      | Surface Carrez du lot 2                                                                       |
| lot_3_numero                       | object       | Numéro du lot 3                                                                               |
| lot_3_surface_carrez               | float64      | Surface Carrez du lot 3                                                                       |
| lot_4_numero                       | object       | Numéro du lot 4                                                                               |
| lot_4_surface_carrez               | float64      | Surface Carrez du lot 4                                                                       |
| lot_5_numero                       | object       | Numéro du lot 5                                                                               |
| lot_5_surface_carrez               | float64      | Surface Carrez du lot 5                                                                       |
| nombre_lots                         | int64        | Nombre de lots                                                                                |
| code_type_local                    | object       | Code de type de local                                                                         |
| type_local                         | object       | Libellé du type de local                                                                      |
| surface_reelle_bati                | float64      | Surface réelle du bâti                                                                        |
| nombre_pieces_principales          | float64      | Nombre de pièces principales                                                                  |
| code_nature_culture                | object       | Code de nature de culture                                                                     |
| nature_culture                      | object       | Libellé de nature de culture                                                                  |
| code_nature_culture_speciale       | object       | Code de nature de culture spéciale                                                            |
| nature_culture_speciale            | object       | Libellé de nature de culture spéciale                                                         |
| surface_terrain                    | float64      | Surface du terrain                                                                            |
| longitude                          | float64      | Longitude du centre de la parcelle concernée (WGS-84)                                         |
| latitude                           | float64      | Latitude du centre de la parcelle concernée (WGS-84)                                          |


In [3]:
#Import des librairies
import pandas as pd
import numpy as np
import sweetviz as sv
import seaborn as sns
from dbfread import DBF
import re
from shapely.geometry import Polygon, Point
import geopandas as gpd

In [4]:
 #Import du fichier csv
df_origin = pd.read_csv("full.csv", low_memory=False)

#Copie du dataframe sur lequel seront faites les étapes de data management
clean_df = df_origin

## Exploration des données

In [6]:
print(clean_df.head(10))
print(f"Ce data set comporte {clean_df.shape[0]} lignes et {clean_df.shape[1]} colonnes ")

  id_mutation date_mutation  numero_disposition nature_mutation  \
0      2024-1    2024-01-02                   1           Vente   
1      2024-2    2024-01-03                   2           Vente   
2      2024-3    2024-01-08                   1           Vente   
3      2024-4    2024-01-03                   1           Vente   
4      2024-4    2024-01-03                   1           Vente   
5      2024-4    2024-01-03                   1           Vente   
6      2024-4    2024-01-03                   1           Vente   
7      2024-5    2024-01-09                   1           Vente   
8      2024-5    2024-01-09                   1           Vente   
9      2024-5    2024-01-09                   1           Vente   

   valeur_fonciere  adresse_numero adresse_suffixe adresse_nom_voie  \
0            346.5             NaN             NaN       LE DELIVRE   
1          10000.0             NaN             NaN   CHEVRY DESSOUS   
2         249000.0             NaN             Na

Faisons un premier tri à plat des données pour vérifier les types de variables, les valeurs aberrantes, le nombre de données manquantes

In [8]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566643 entries, 0 to 1566642
Data columns (total 40 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   id_mutation                   1566643 non-null  object 
 1   date_mutation                 1566643 non-null  object 
 2   numero_disposition            1566643 non-null  int64  
 3   nature_mutation               1566643 non-null  object 
 4   valeur_fonciere               1549589 non-null  float64
 5   adresse_numero                946773 non-null   float64
 6   adresse_suffixe               67087 non-null    object 
 7   adresse_nom_voie              1558411 non-null  object 
 8   adresse_code_voie             1558442 non-null  object 
 9   code_postal                   1558158 non-null  float64
 10  code_commune                  1566643 non-null  object 
 11  nom_commune                   1566643 non-null  object 
 12  code_departement            

In [9]:
#Génération d'un rapport intéractif html permettant de décrire les données avant data management
#rapport = sv.analyze(clean_df)
#rapport.show_html("rapport_before_datamgmt.html")

## Nettoyage de la base de données

Suppression des colonnes non utilisées dès le début du data management : ancien_id_parcelle, ancien_code_commune, ancien_nom_commune

In [12]:
clean_df= clean_df.drop(columns=['ancien_code_commune', 'ancien_nom_commune', 'ancien_id_parcelle'])

Suppression des doublons

In [14]:
clean_df.drop_duplicates(subset=None, keep='first', inplace=True)

In [15]:
clean_df["nature_mutation"].unique()

array(['Vente', "Vente en l'état futur d'achèvement",
       'Vente terrain à bâtir', 'Echange', 'Adjudication',
       'Expropriation'], dtype=object)

Les mutations ayant pour nature: "Adjudication", "Expropriation" et "Echange' ne nous intéressent pas ici. Elles risquent en plus de biaiser nos analyses. Nous allons donc garder uniquement les lignes ayant pour nature de mutation : 'Vente', "Vente en l'état futur d'achèvement",'Vente terrain à bâtir'

In [17]:
#Garder uniquement les mutations : 'Vente', "Vente en l'état futur d'achèvement",'Vente terrain à bâtir'

clean_df = clean_df[(clean_df["nature_mutation"]=="Vente") | (clean_df["nature_mutation"]=="Vente en l'état futur d'achèvement") | 
(clean_df["nature_mutation"]=="Vente terrain à bâtir")]
#Remarque: la colonne nature_mutation ne contient pas de donnée manquante

#Vérification
print(clean_df["nature_mutation"].unique())

['Vente' "Vente en l'état futur d'achèvement" 'Vente terrain à bâtir']


Les lignes n'ayant ni la surface réelle bâtie ni la surface du terrain sont également supprimés car dans ce cas le prix au m² ne pourra pas être calculé et les valeurs foncières seules n'auront pas de sens

In [19]:
#Supprimer lignes qui n'ont ni la surface réelle bâtie ni la surface terrain

clean_df = clean_df.dropna(axis=0, subset=["surface_reelle_bati","surface_terrain"], how="all")
#how = "all" permet de spécifier que la donnée doit être manquante dans les deux colonnes pour que la ligne soit supprimée

Nous avons décidé de travailler uniquement en France métropolitaine. Nous excluons donc les départements d'outre-mer

In [21]:
clean_df["code_departement"].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '69', '70', '71', '72', '73', '74', '76', '77', '78', '79', '80',
       '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91',
       '92', '93', '94', '95', '971', '972', '973', '974', '75'],
      dtype=object)

In [22]:
#Suppression des lignes avec un département d'outre mer

clean_df = clean_df[(clean_df["code_departement"] != "971") & (clean_df["code_departement"] != "972") & (clean_df["code_departement"] != "973") &
(clean_df["code_departement"] != "974")]

print(clean_df["code_departement"].unique())

print(f"Le nombre de lignes de ce data frame est maintenant de {clean_df.shape[0]}")

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '2A' '2B' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55'
 '56' '58' '59' '60' '61' '62' '63' '64' '65' '66' '69' '70' '71' '72'
 '73' '74' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86' '87'
 '88' '89' '90' '91' '92' '93' '94' '95' '75']
Le nombre de lignes de ce data frame est maintenant de 1156267


## Conversion de variables
Pour certaines variables, le type de variable n'est pas le bon. 

### Conversion de la variable date_mutation en format date

In [25]:
clean_df["date_mutation"] = pd.to_datetime(clean_df["date_mutation"])

### Conversion des variables numériques non quantitatives en variables catégorielles
Certaines variables sont numériques (integer ou float) mais ne sont pas quantitatives ie qu'il n'y a pas de sens à calculer des moyennes, écarts-types pour ces variables. On les convertit donc en object

In [27]:
#Conversion des variables numériques (integer ou float) non quantitatives en object :

list_var = ["numero_disposition", "adresse_numero", "code_postal",  "code_type_local", "lot5_numero"]

for i in list_var: 
    clean_df[i] = clean_df[i].astype("object")

## Gestion des données manquantes

In [29]:
#Connaître le pourcentage de données manquantes par colonne
print(clean_df.isnull().sum())
print(((clean_df.isnull().sum())/len(clean_df))*100)

id_mutation                           0
date_mutation                         0
numero_disposition                    0
nature_mutation                       0
valeur_fonciere                   12882
adresse_numero                   559591
adresse_suffixe                 1114470
adresse_nom_voie                     25
adresse_code_voie                     0
code_postal                          98
code_commune                          0
nom_commune                           0
code_departement                      0
id_parcelle                           0
numero_volume                   1156267
lot1_numero                      973915
lot1_surface_carrez             1066383
lot2_numero                     1083099
lot2_surface_carrez             1133777
lot3_numero                     1144870
lot3_surface_carrez             1154086
lot4_numero                     1152517
lot4_surface_carrez             1155735
lot5_numero                     1154640
lot5_surface_carrez             1156097


### Code postale
Le code postale contient 8460 données manquantes (0.58%). Nous allons vérifier si les adresses avec le même code commune ont le même code postale

In [31]:
#Création d'une colonne code_postal_nb qui compte le nombre de codes postaux par groupe de code commune
clean_df["code_postal_nb"] = clean_df.groupby("code_commune")["code_postal"].transform("nunique")
clean_df[["code_commune","code_postal", "code_postal_nb"]][clean_df["code_postal_nb"] > 1]

,code_commune,code_postal,code_postal_nb
834,01104,1410.0,2
835,01104,1410.0,2
836,01104,1410.0,2
837,01104,1410.0,2
1349,01104,1410.0,2
...,...,...,...
1513451,95500,95000.0,2
1513470,95500,95300.0,2
1513471,95500,95300.0,2
1513472,95500,95300.0,2


On comprend que deux adresses avec le même code commune peuvent avoir des codes postaux différents. Ainsi il n'est pas possible de récupérer le code postale à partir du code commune.

### Valeur foncière
La variable "valeur_fonciere" comporte moins de 1% de données manquantes (N=14 209). Explorons le lien entre valeur foncière et id_mutation.

In [34]:
#Colonne "valeur_fonciere_par_id" pour compter le nombre de valeur foncière par id_mutation
clean_df["valeur_fonciere_par_id"] = clean_df.groupby("id_mutation")["valeur_fonciere"].transform("nunique")

#Afficher les lignes qui ont plus d'une valeur foncière par id_mutation
clean_df[["id_mutation","valeur_fonciere", "valeur_fonciere_par_id"]][clean_df["valeur_fonciere_par_id"] > 1]

,id_mutation,valeur_fonciere,valeur_fonciere_par_id


Ainsi chaque id_mutation est associée à une seule et unique valeur foncière. Les valeurs foncières manquantes correspondent donc peut-être à des données **confidentielles**. Nous avons donc fait le choix de ne pas les imputer et de supprimer du jeu de données les lignes avec une valeur foncière manquante étant donné en plus leur faible nombre (<1% des données, n=14 209) et leur non utilisabilité

In [36]:
##Supprimer les lignes où 'valeur_fonciere' est manquante
clean_df = clean_df.dropna(subset=['valeur_fonciere'])

### Longitude et latitude

Ces variables comportent un peu plus de 1% de données manquantes. On ne va pas les imputer, elles ne seront juste pas utilisée pour les visualisations cartographiques mais on garde ces lignes dans le jeu de données

## Création de nouvelles variables

### Variable "nom_departement" et "nom_region"

La base de données utilisés peut être retrouvé sur : https://www.data.gouv.fr/fr/datasets/departements-de-france/ 

In [40]:
#Importer la base de référence "liste des départements de France"
df_reference_depart = pd.read_csv('departements-france.csv', sep=',', encoding='utf-8')
print(df_reference_depart.head())

#Renommer la colonne code_departement de la base de référence pour éviter les conflits
df_reference_depart.rename(columns={'code_departement': 'code_departement_ref'}, inplace=True)


  code_departement          nom_departement  code_region  \
0               01                      Ain           84   
1               02                    Aisne           32   
2               03                   Allier           84   
3               04  Alpes-de-Haute-Provence           93   
4               05             Hautes-Alpes           93   

                   nom_region  
0        Auvergne-Rhône-Alpes  
1             Hauts-de-France  
2        Auvergne-Rhône-Alpes  
3  Provence-Alpes-Côte d'Azur  
4  Provence-Alpes-Côte d'Azur  


In [41]:
#Affichage des colonnes de la base de référence et notre jeu de données
print(df_reference_depart.columns)
print(clean_df.columns)

Index(['code_departement_ref', 'nom_departement', 'code_region', 'nom_region'], dtype='object')
Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'numero_volume',
       'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'code_postal_nb', 'valeur_fonciere_par_id'],
      dtype='object')


In [42]:
#Jointure entre notre base de données et la base de référence pour obtenir les noms des départements

clean_df = clean_df.merge(
    df_reference_depart[['code_departement_ref', 'nom_departement', 'nom_region']],  # Colonnes nécessaires
    left_on='code_departement',  # Clé de jointure dans clean_df
    right_on='code_departement_ref',  # Clé de jointure dans df_reference_depart
    how='left'  # Jointure à gauche pour conserver toutes les lignes de clean_df
)

# Afficher les premières lignes pour vérifier les colonnes ajoutées
print(clean_df.head())

  id_mutation date_mutation numero_disposition nature_mutation  \
0      2024-1    2024-01-02                  1           Vente   
1      2024-2    2024-01-03                  2           Vente   
2      2024-3    2024-01-08                  1           Vente   
3      2024-4    2024-01-03                  1           Vente   
4      2024-5    2024-01-09                  1           Vente   

   valeur_fonciere adresse_numero adresse_suffixe adresse_nom_voie  \
0            346.5            NaN             NaN       LE DELIVRE   
1          10000.0            NaN             NaN   CHEVRY DESSOUS   
2         249000.0            NaN             NaN       PIN HAMEAU   
3         329500.0           29.0             NaN       PL DU JURA   
4          20000.0            NaN             NaN        AU CHAUME   

  adresse_code_voie code_postal  ... code_nature_culture_speciale  \
0              B020      1230.0  ...                          NaN   
1              B007      1170.0  ...        

In [43]:
#Vérification de nombre de lignes non affectées

lignes_non_affectees = clean_df[clean_df['nom_departement'].isnull() | clean_df['nom_region'].isnull()]
print(f"Le nombre de lignes pourlesquelles un nom de département ou de région n'a pas pu être affecté est de {len(lignes_non_affectees)}")

Le nombre de lignes pourlesquelles un nom de département ou de région n'a pas pu être affecté est de 0


In [44]:
#Suppresion de la colonne code_departement_ref

clean_df = clean_df.drop(columns = "code_departement_ref")

### Variable diagonale du vide ("dans_diag_vide")

Cette variable indique si l'adresse de la transaction est situé dans la diagonale du vide ou pas. Il s'agit d'une variable booléenne (True/False)

In [46]:
diagonale_du_vide = Polygon([
    (5.5, 50.5),  # Point 1zone
    (5.7, 50),   # Point 2
    (2.5, 42.0),   # Point 3
    (-1.5, 43.0),  # Point 4
    (-0.35, 44.7),
    (2.45, 48.0),
    (4.0, 49.5),
    (5.5, 50.5)# Retour au Point 1
])

# Fonction pour vérifier si un point est dans la diagonale du vide
def est_dans_diagonale(lon, lat):
    point = Point(lon, lat)
    return diagonale_du_vide.contains(point)

# Application sur le DataFrame
clean_df['dans_diag_vide'] = clean_df.apply(lambda row: est_dans_diagonale(row['longitude'], row['latitude']), axis=1)

clean_df.head(10)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,code_postal_nb,valeur_fonciere_par_id,nom_departement,nom_region,dans_diag_vide
0,2024-1,2024-01-02,1,Vente,346.5,NaN,NaN,LE DELIVRE,B020,1230.0,...,NaN,NaN,99.0,5.530952,45.952439,1,1,Ain,Auvergne-Rhône-Alpes,False
1,2024-2,2024-01-03,2,Vente,10000.0,NaN,NaN,CHEVRY DESSOUS,B007,1170.0,...,NaN,NaN,115.0,6.043339,46.282256,1,1,Ain,Auvergne-Rhône-Alpes,False
2,2024-3,2024-01-08,1,Vente,249000.0,NaN,NaN,PIN HAMEAU,B086,1290.0,...,NaN,NaN,497.0,4.911143,46.247235,1,1,Ain,Auvergne-Rhône-Alpes,False
3,2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,...,NaN,NaN,NaN,6.058695,46.332212,1,1,Ain,Auvergne-Rhône-Alpes,False
4,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,AU CHAUME,B579,1110.0,...,NaN,NaN,1584.0,5.548635,45.900606,1,1,Ain,Auvergne-Rhône-Alpes,False
5,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,LES FATES,B606,1110.0,...,NaN,NaN,188.0,5.555398,45.899514,1,1,Ain,Auvergne-Rhône-Alpes,False
6,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,LES FATES,B606,1110.0,...,NaN,NaN,2532.0,5.554337,45.899235,1,1,Ain,Auvergne-Rhône-Alpes,False
7,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,EN CHANAUD,B574,1110.0,...,NaN,NaN,2615.0,5.540564,45.892555,1,1,Ain,Auvergne-Rhône-Alpes,False
8,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,LA BERROTIERE,B562,1110.0,...,NaN,NaN,1890.0,5.546941,45.886455,1,1,Ain,Auvergne-Rhône-Alpes,False
9,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,RADIOT,B644,1110.0,...,NaN,NaN,926.0,5.541926,45.887399,1,1,Ain,Auvergne-Rhône-Alpes,False


### Variable zone sismique

Cette variable est une variable catégorielle indiquant la zone sismique de la localité (5 catégories: sismicité très faible, faible, moyenne, modérée et forte). Nous avons utilisé une base de données de référence sur le zonage sismique en France métrpolitaine disponible sur : https://www.data.gouv.fr/fr/datasets/zonage-sismique-de-la-france-1/#/information 

In [48]:
#Chargement du fichier dbf
data_dbf = DBF('France_zonage_sismique.dbf', encoding='latin1')
df_sismique = pd.DataFrame(data_dbf)

In [49]:
df_sismique.head()

,insee,nom,departemen,Sismicite
0,64485,Saint-Jean-Pied-de-Port,Pyrenees-Atlantiques,4 - Moyenne
1,64166,çaro,Pyrenees-Atlantiques,4 - Moyenne
2,64155,Bustince-Iriberry,Pyrenees-Atlantiques,4 - Moyenne
3,64297,Lacarre,Pyrenees-Atlantiques,4 - Moyenne
4,64247,Gotein-Libarrenx,Pyrenees-Atlantiques,4 - Moyenne


In [50]:
#Le code insee correspond au code commune on peut donc merger avec notre base de données en utilisant cette clé

clean_df_merged = pd.merge(clean_df, df_sismique, left_on="code_commune", right_on = "insee", how="left")

#Affichage du data frame après jointure 
clean_df_merged[["code_commune", "insee", "nom", "nom_commune", "Sismicite"]]

,code_commune,insee,nom,nom_commune,Sismicite
0,01076,01076,Chaley,Chaley,3 - Modérée
1,01103,01103,Chevry,Chevry,3 - Modérée
2,01203,01203,Laiz,Laiz,2 - Faible
3,01173,01173,Gex,Gex,3 - Modérée
4,01185,01185,Hauteville-Lompnes,Plateau d'Hauteville,3 - Modérée
...,...,...,...,...,...
1143380,75114,NaN,NaN,Paris 14e Arrondissement,NaN
1143381,75116,NaN,NaN,Paris 16e Arrondissement,NaN
1143382,75111,NaN,NaN,Paris 11e Arrondissement,NaN
1143383,75114,NaN,NaN,Paris 14e Arrondissement,NaN


In [51]:
#Nombre de données manquantes sur l'insee suite à la jointure (lignes pour lesquelles la jointure n'a pas pu être faite
print(f"Le nombre de données manquantes pour la colonne insee est de :{clean_df_merged['insee'].isnull().sum()}. C'est le nombre de lignes pourlesquelles la jointure n'a pas pu être faite")

Le nombre de données manquantes pour la colonne insee est de :28090. C'est le nombre de lignes pourlesquelles la jointure n'a pas pu être faite


In [52]:
#Affichage des noms de communes pour lesquelles où la jointure n'a pas pu être faite pour comprendre : 
print(clean_df_merged["nom_commune"][clean_df_merged["insee"].isnull()].unique())

['Marseille 15e Arrondissement' 'Marseille 7e Arrondissement'
 'Marseille 6e Arrondissement' 'Marseille 12e Arrondissement'
 'Marseille 8e Arrondissement' 'Marseille 9e Arrondissement'
 'Marseille 5e Arrondissement' 'Marseille 4e Arrondissement'
 'Marseille 10e Arrondissement' 'Marseille 3e Arrondissement'
 'Marseille 1er Arrondissement' 'Marseille 14e Arrondissement'
 'Marseille 13e Arrondissement' 'Marseille 2e Arrondissement'
 'Marseille 11e Arrondissement' 'Marseille 16e Arrondissement'
 'Lyon 9e Arrondissement' 'Lyon 4e Arrondissement'
 'Lyon 1er Arrondissement' 'Lyon 2e Arrondissement'
 'Lyon 5e Arrondissement' 'Lyon 8e Arrondissement'
 'Lyon 7e Arrondissement' 'Lyon 6e Arrondissement'
 'Lyon 3e Arrondissement' 'Paris 20e Arrondissement'
 'Paris 18e Arrondissement' 'Paris 10e Arrondissement'
 'Paris 3e Arrondissement' 'Paris 19e Arrondissement'
 'Paris 2e Arrondissement' 'Paris 17e Arrondissement'
 'Paris 9e Arrondissement' 'Paris 8e Arrondissement'
 'Paris 4e Arrondissement' 'Pa

Il semble que les lignes pour lesquelles la jointure n'a pas pu être faite sont les lignes correspondant à des villes avec arrondissements(Paris, Lyon, Marseille). Résolvons donc ce problème 

In [54]:
#Simplifier la colonne "nom_commune" en ne prenant que le nom ville sans l'arrondissement 
clean_df_merged["nom_commune_reg"]= clean_df_merged['nom_commune'][clean_df_merged["insee"].isnull()].apply(lambda x: re.sub(r'\s\d+[a-zA-Z]+\sArrondissement', '', x))
#La transformation n'est faite que sur les lignes avec l'insee manquant pour diminuer le nombre d'itérations
#Fonction sub de regex : permet de remplacer les suffixes "1er arrondissement" etc par des chaînes vides. On ne garde que le nom de la ville

#Vérifications : 
print(clean_df_merged[["nom_commune_reg", "nom_commune"]][clean_df_merged["insee"].isnull()])
print(clean_df_merged["nom_commune_reg"][clean_df_merged["insee"].isnull()].unique()) #Les valeurs uniques ne comportent plus le suffixe avec arrondissement pour cette colonne

        nom_commune_reg                   nom_commune
122535        Marseille  Marseille 15e Arrondissement
122536        Marseille   Marseille 7e Arrondissement
122537        Marseille   Marseille 6e Arrondissement
122538        Marseille  Marseille 12e Arrondissement
122539        Marseille   Marseille 8e Arrondissement
...                 ...                           ...
1143380           Paris      Paris 14e Arrondissement
1143381           Paris      Paris 16e Arrondissement
1143382           Paris      Paris 11e Arrondissement
1143383           Paris      Paris 14e Arrondissement
1143384           Paris      Paris 15e Arrondissement

[28090 rows x 2 columns]
['Marseille' 'Lyon' 'Paris']


In [55]:
#Compléter la jointure en utilisant la colonne nom_commune_reg pour les villes avec arrondissement n'ayant pas pu être jointes

#clean_df_villes comporte les villes avec arrondissement n'ayant pas pu être jointes 
clean_df_villes = clean_df_merged[(clean_df_merged["insee"].isnull())]

#Enlever les colonnes "insee", "nom", "departemen" et "Sismicite" de df_villes pour ne pas avoir de doublons lors de la jointure

clean_df_villes = clean_df_villes.drop(columns=["insee", "nom", "departemen", "Sismicite"])

#Jointure entre df_villes et df_sismiques en utilisant la colonne nom_commune_reg de notre base de données et la colonne "nom" de la base de référence
clean_df_merged_villes = pd.merge(clean_df_villes, df_sismique, left_on="nom_commune_reg", right_on = "nom", how="left")

#Vérification du nombre de lignes non jointes : 

print(f"Le nombre de lignes non jointes est de :{clean_df_merged_villes['insee'].isnull().sum()}. Les villes avec arrondissement ont bien pu être jointes")

Le nombre de lignes non jointes est de :0. Les villes avec arrondissement ont bien pu être jointes


In [56]:
#Sous_dataframe de df_merged qui ne contient que les lignes avec les villes sans arrondissement (ie toutes les communes sauf Paris, Lyon et Marseille)
clean_df_merged_sans_villes = clean_df_merged[(clean_df_merged["nom_commune_reg"] != "Paris") & (clean_df_merged["nom_commune_reg"] != "Lyon") & (clean_df_merged["nom_commune_reg"] != "Marseille")]

print(clean_df_merged_sans_villes.columns)
print(clean_df_merged_villes.columns)
#Les deux df ont les mêmes colonnes

# Concaténer verticalement (ajout de lignes) clean_df_merged_sans_villes avec clean_df_merged_villes 
clean_df_merged_final = pd.concat([clean_df_merged_sans_villes, clean_df_merged_villes], axis=0)

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'numero_volume',
       'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'code_postal_nb', 'valeur_fonciere_par_id', 'nom_departement',
       'nom_region', 'dans_diag_vide', 'insee', 'nom', 'departemen',
       'Sismicite', 'nom_commune_reg'],
      dtype='object')
Index(['id_mutation'

In [57]:
#Faire un reset index
clean_df_merged_final = clean_df_merged_final.reset_index(drop=True)

#Supprimer les colonnes "insee", "nom", "departemen" et "nom_commune_reg"

clean_df_merged_final.drop(columns=["insee", "nom", "departemen", "nom_commune_reg"], inplace=True)

#Affichage des premières lignes du data frame avec la variable sismicité
clean_df_merged_final.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,nature_culture_speciale,surface_terrain,longitude,latitude,code_postal_nb,valeur_fonciere_par_id,nom_departement,nom_region,dans_diag_vide,Sismicite
0,2024-1,2024-01-02,1,Vente,346.5,NaN,NaN,LE DELIVRE,B020,1230.0,...,NaN,99.0,5.530952,45.952439,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée
1,2024-2,2024-01-03,2,Vente,10000.0,NaN,NaN,CHEVRY DESSOUS,B007,1170.0,...,NaN,115.0,6.043339,46.282256,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée
2,2024-3,2024-01-08,1,Vente,249000.0,NaN,NaN,PIN HAMEAU,B086,1290.0,...,NaN,497.0,4.911143,46.247235,1,1,Ain,Auvergne-Rhône-Alpes,False,2 - Faible
3,2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,...,NaN,NaN,6.058695,46.332212,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée
4,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,AU CHAUME,B579,1110.0,...,NaN,1584.0,5.548635,45.900606,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée


## Variable zone inondable

Cette variable indique si le bien immobilier est situé dans une zone innondable ou non. Une autre variable "nature_risque" est également crée pour connaitre la nature du risque d'innondation (Débordement ou submersion marine). La base utilisé peut être retrouvé sur : https://vdugrain.carto.com/tables/n_inondable/public

In [59]:
#Import du fichier geojson
inondation = gpd.read_file('n_inondable.geojson')

In [60]:
inondation.head()

,cartodb_id,uuid,id,scenario,id_tri,nom_tri,nature_risque,url_doc_tri,scenario_libelle,geometry
0,216,273bfc6f-594e-4ad1-8642-2f539e12383f,373716,01For,FRD_TRI_MONTPELLIER,Montpellier - Lunel - Mauguio - Palavas-les-Flots,Débordement,https://www.rhone-mediterranee.eaufrance.fr/ca...,Scénario fréquent (probabilité forte) : événem...,"MULTIPOLYGON (((3.88328 43.50619, 3.88311 43.5..."
1,154,708d6287-28c4-4799-b3fc-1d8e8ea56e07,394315,01For,FRL_TRI_SAINT_JOSEPH_DEB_CE,Zone urbanisée de la commune de St-Joseph conc...,Débordement,https://www.reunion.developpement-durable.gouv...,Scénario fréquent (probabilité forte) : événem...,"MULTIPOLYGON (((55.60316 -21.38149, 55.60316 -..."
2,155,94043af0-d9cb-402f-ba3b-673409b9acd9,394312,01For,FRL_TRI_PLANEZE_TAMPON_SAINT_PIERRE_DEB_CE,Zones urbanisées des communes de Saint-Pierre ...,Débordement,https://www.reunion.developpement-durable.gouv...,Scénario fréquent (probabilité forte) : événem...,"MULTIPOLYGON (((55.4455 -21.32528, 55.44576 -2..."
3,588,1029667f-f06d-45c0-9c02-787777e3cce9,261293,04Fai,FRG_TRI_ST_NAZAIRE_PRESQU_ILE_DE_GUERANDE,Saint-Nazaire - Presqu'Île de Guérande,Submersion marine,https://www.centre-val-de-loire.developpement-...,Scénario extrême (probabilité faible) : phénom...,"MULTIPOLYGON (((-2.50528 47.28264, -2.50499 47..."
4,495,895dddb3-d48c-4f62-a3f5-afdc4d796bba,276031,01For,FRD_TRI_NARBONNE,Narbonne,Submersion marine,https://www.rhone-mediterranee.eaufrance.fr/ca...,Scénario fréquent (probabilité forte) : événem...,"MULTIPOLYGON (((2.99244 43.03341, 2.99269 43.0..."


In [61]:
 #Utiliser clean_df_merged_final (DataFrame classique des propriétés)
clean_df_merged_final['geometry'] = clean_df_merged_final.apply(
    lambda row: Point(row['longitude'], row['latitude']), axis=1
)

# Convertir en GeoDataFrame (nécessaire pour traiter la colonne geometry)
gdf_proprietes = gpd.GeoDataFrame(clean_df_merged_final, geometry='geometry', crs="EPSG:4326")

# Vérifier que les systèmes de coordonnées sont alignés
if gdf_proprietes.crs != inondation.crs:
    inondation = inondation.to_crs(gdf_proprietes.crs)

# Effectuer la jointure spatiale
clean_df_final = gpd.sjoin(gdf_proprietes, inondation, how="left", predicate="within")

# Ajouter une colonne True/False indiquant si la propriété est dans une zone inondable
clean_df_final['in_zone_inondable'] = clean_df_final['index_right'].notna()


clean_df_final.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,cartodb_id,uuid,id,scenario,id_tri,nom_tri,nature_risque,url_doc_tri,scenario_libelle,in_zone_inondable
0,2024-1,2024-01-02,1,Vente,346.5,NaN,NaN,LE DELIVRE,B020,1230.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2024-2,2024-01-03,2,Vente,10000.0,NaN,NaN,CHEVRY DESSOUS,B007,1170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2024-3,2024-01-08,1,Vente,249000.0,NaN,NaN,PIN HAMEAU,B086,1290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,AU CHAUME,B579,1110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [62]:
# Compter le nombre de propriétés dans des zones inondables
nb_inondables = clean_df_final['in_zone_inondable'].sum()

print(f"Nombre de propriétés dans des zones inondables : {nb_inondables}")


Nombre de propriétés dans des zones inondables : 606


In [63]:
#Supprimer les colonnes inutiles
colonnesasup = ['cartodb_id', 'uuid', 'id', 'scenario', 'id_tri', 'nom_tri', 'url_doc_tri', 'scenario_libelle', 'geometry', 'index_right']
clean_df_final.drop(columns = colonnesasup, inplace = True)

clean_df_final.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,longitude,latitude,code_postal_nb,valeur_fonciere_par_id,nom_departement,nom_region,dans_diag_vide,Sismicite,nature_risque,in_zone_inondable
0,2024-1,2024-01-02,1,Vente,346.5,NaN,NaN,LE DELIVRE,B020,1230.0,...,5.530952,45.952439,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée,NaN,False
1,2024-2,2024-01-03,2,Vente,10000.0,NaN,NaN,CHEVRY DESSOUS,B007,1170.0,...,6.043339,46.282256,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée,NaN,False
2,2024-3,2024-01-08,1,Vente,249000.0,NaN,NaN,PIN HAMEAU,B086,1290.0,...,4.911143,46.247235,1,1,Ain,Auvergne-Rhône-Alpes,False,2 - Faible,NaN,False
3,2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,...,6.058695,46.332212,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée,NaN,False
4,2024-5,2024-01-09,1,Vente,20000.0,NaN,NaN,AU CHAUME,B579,1110.0,...,5.548635,45.900606,1,1,Ain,Auvergne-Rhône-Alpes,False,3 - Modérée,NaN,False


### Variable prix/m²

Cette variable est une variable quantitative. Elle indique prix au m² selon que le bien soit une maison, un appartement ou une dépendance. 

In [65]:
# Créer des colonnes intermédiaires  total_surface_reelle_batie et total_superficie_terrain
clean_df_final['total_surface_reelle_batie'] = clean_df_final.groupby('id_mutation')['surface_reelle_bati'].transform('sum')
clean_df_final['total_superficie_terrain'] = clean_df_final.groupby('id_mutation')['surface_terrain'].transform('sum')

In [66]:
# Agréger les données
aggregated_data = clean_df_final.groupby('id_mutation').agg({
    'valeur_fonciere': 'first',
    'total_surface_reelle_batie': 'first',
    'total_superficie_terrain': 'first',
    'type_local': lambda x: ','.join([str(i) for i in x.unique()])  # Regrouper les types locaux uniques
}).reset_index()

# Condition 1 : Combinaison complexe de types locaux
def calcul_prix_par_m2(row):
    # Nettoyer les espaces vides ou 'nan' et transformer en liste
    types_locaux = set(filter(lambda x: x not in ['nan', ''], row['type_local'].split(',')))  # Supprimer les valeurs 'nan' et les chaînes vides
    prix_par_m2 = np.nan
    
    # Vérifier si les types locaux sont dans la liste complexe
    types_complexes = {"Appartement", "Maison", "Dépendance", "Local industriel. commercial ou assimilé"}
    
    if types_locaux.issubset(types_complexes):
        if row['total_surface_reelle_batie'] > 0:
            prix_par_m2 = row['valeur_fonciere'] / row['total_surface_reelle_batie']
        elif row['total_superficie_terrain'] > 0:
            prix_par_m2 = row['valeur_fonciere'] / row['total_superficie_terrain']
    return prix_par_m2

aggregated_data['prix_m²'] = aggregated_data.apply(calcul_prix_par_m2, axis=1)

# Condition 2 : Appartement ou Appartement et Dépendance
apparts_dependances = aggregated_data['type_local'].str.contains('Appartement', na=False) & ~aggregated_data['type_local'].str.contains('Maison', na=False)

aggregated_data.loc[apparts_dependances, 'prix_m²'] = np.where(
    aggregated_data.loc[apparts_dependances, 'total_surface_reelle_batie'] > 0,
    aggregated_data.loc[apparts_dependances, 'valeur_fonciere'] / aggregated_data.loc[apparts_dependances, 'total_surface_reelle_batie'],
    np.nan  # Remplacer "Données non fournies" par NaN ==> ?? revoir
)

# Condition 3 : Maison ou Maison et Dépendance
maisons_dependances = aggregated_data['type_local'].str.contains('Maison', na=False)

aggregated_data.loc[maisons_dependances, 'prix_m²'] = np.where(
    aggregated_data.loc[maisons_dependances, 'total_surface_reelle_batie'] > 0,
    aggregated_data.loc[maisons_dependances, 'valeur_fonciere'] / aggregated_data.loc[maisons_dependances, 'total_surface_reelle_batie'],
    np.where(
        aggregated_data.loc[maisons_dependances, 'total_superficie_terrain'] > 0,
        aggregated_data.loc[maisons_dependances, 'valeur_fonciere'] / aggregated_data.loc[maisons_dependances, 'total_superficie_terrain'],
        np.nan  # Remplacer les valeurs manquantes par NaN
    )
)

# Conversion de la colonne 'prix_m²' en float (au cas où il y aurait encore des chaînes)
aggregated_data['prix_m²'] = pd.to_numeric(aggregated_data['prix_m²'], errors='coerce')

# Affichage du résultat pour vérifier
print(aggregated_data[['id_mutation', 'type_local', 'prix_m²']].head(10))


   id_mutation             type_local      prix_m²
0       2024-1                    nan     3.500000
1      2024-10                 Maison  1727.272727
2     2024-100  nan,Dépendance,Maison  1202.830189
3    2024-1000            Appartement  3000.000000
4   2024-10000                    nan     0.500079
5  2024-100000      Dépendance,Maison  3159.042553
6  2024-100001             Maison,nan  1421.052632
7  2024-100003            Appartement  3351.063830
8  2024-100004             Maison,nan   576.923077
9  2024-100005            Appartement  1512.500000


In [67]:
# Condition 4 : Local industriel. commercial ou assimilé

locaux_commerciaux = aggregated_data['type_local'].str.contains('Local industriel. commercial ou assimilé', na=False)

aggregated_data.loc[locaux_commerciaux, 'prix_m²'] = np.where(
    aggregated_data.loc[locaux_commerciaux, 'total_surface_reelle_batie'] > 0,
    aggregated_data.loc[locaux_commerciaux, 'valeur_fonciere'] / aggregated_data.loc[locaux_commerciaux, 'total_surface_reelle_batie'],
    np.where(
        aggregated_data.loc[locaux_commerciaux, 'total_superficie_terrain'] > 0,
        aggregated_data.loc[locaux_commerciaux, 'valeur_fonciere'] / aggregated_data.loc[locaux_commerciaux, 'total_superficie_terrain'],
        np.nan  # Remplacer les valeurs manquantes par NaN
    )
)
# Conversion de la colonne 'prix_m²' en float (au cas où il y aurait encore des chaînes)
aggregated_data['prix_m²'] = pd.to_numeric(aggregated_data['prix_m²'], errors='coerce')
# Affichage du résultat pour vérifier
print(aggregated_data[['id_mutation', 'type_local', 'prix_m²']].head(10))

   id_mutation             type_local      prix_m²
0       2024-1                    nan     3.500000
1      2024-10                 Maison  1727.272727
2     2024-100  nan,Dépendance,Maison  1202.830189
3    2024-1000            Appartement  3000.000000
4   2024-10000                    nan     0.500079
5  2024-100000      Dépendance,Maison  3159.042553
6  2024-100001             Maison,nan  1421.052632
7  2024-100003            Appartement  3351.063830
8  2024-100004             Maison,nan   576.923077
9  2024-100005            Appartement  1512.500000


In [68]:
# Vérifier le nombre de valeurs manquantes dans 'prix_m²'
nan_count = aggregated_data['prix_m²'].isna().sum()
print(f"Nombre de NaN dans 'prix_m²' : {nan_count}")

Nombre de NaN dans 'prix_m²' : 0


In [69]:
# Ajouter la colonne 'prix_m²' à 'clean_df' en faisant une jointure sur 'id_mutation'
clean_df_final = clean_df_final.merge(aggregated_data[['id_mutation', 'prix_m²']], on='id_mutation', how='left')

# Afficher les premières lignes pour vérifier que 'prix_m²' a bien été ajouté
print(clean_df_final[['id_mutation', 'date_mutation', 'nature_mutation', 'type_local', 'prix_m²']].head(10))

  id_mutation date_mutation nature_mutation   type_local      prix_m²
0      2024-1    2024-01-02           Vente          NaN     3.500000
1      2024-2    2024-01-03           Vente          NaN    86.956522
2      2024-3    2024-01-08           Vente          NaN   501.006036
3      2024-4    2024-01-03           Vente  Appartement  3702.247191
4      2024-5    2024-01-09           Vente          NaN     0.096911
5      2024-5    2024-01-09           Vente          NaN     0.096911
6      2024-5    2024-01-09           Vente          NaN     0.096911
7      2024-5    2024-01-09           Vente          NaN     0.096911
8      2024-5    2024-01-09           Vente          NaN     0.096911
9      2024-5    2024-01-09           Vente          NaN     0.096911


In [70]:
#Afficher le min et le max du prix/m²
print(f"Le prix/m² minimal est de {clean_df_final["prix_m²"].min()} euros par m²")
print(f"Le prix/m² maximal est de {clean_df_final["prix_m²"].max()} euros par m²")

Le prix/m² minimal est de 2.469709018883395e-06 euros par m²
Le prix/m² maximal est de 697248.44 euros par m²


Certaines valeurs du prix/m² nous paraissent aberrantes. Nous allons appliquer un filtre

In [72]:
## Filtrage des prix réalistes
prix_min = 500   # Prix minimum réaliste en €/m²
prix_max = 25000 # Prix maximum réaliste en €/m²
# Créer un filtre pour les prix réalistes
df_filtre_final = clean_df_final[(clean_df_final['prix_m²'] >= prix_min) & (clean_df_final['prix_m²'] <= prix_max)]

# Afficher un résumé des données filtrées
print(f"Le data frame comporte désormais {df_filtre_final.shape[0]} lignes")
print(df_filtre_final.describe())


Le data frame comporte désormais 671188 lignes
                       date_mutation  valeur_fonciere  lot1_surface_carrez  \
count                         671188     6.711880e+05         88273.000000   
mean   2024-04-06 02:03:49.606012160     4.694731e+05            58.956349   
min              2024-01-02 00:00:00     5.600000e+02             1.000000   
25%              2024-02-21 00:00:00     1.260000e+05            34.400000   
50%              2024-04-08 00:00:00     2.050000e+05            52.400000   
75%              2024-05-24 00:00:00     3.400000e+05            71.780000   
max              2024-06-30 00:00:00     1.180268e+08          4759.970000   
std                              NaN     2.588866e+06            56.802014   

       lot2_surface_carrez  lot3_surface_carrez  lot4_surface_carrez  \
count         22125.000000          2084.000000           483.000000   
mean             62.350590            71.985130            87.355238   
min               0.600000        

In [73]:
df_filtre_final.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'numero_volume',
       'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'code_postal_nb', 'valeur_fonciere_par_id', 'nom_departement',
       'nom_region', 'dans_diag_vide', 'Sismicite', 'nature_risque',
       'in_zone_inondable', 'total_surface_reelle_batie',
       'total_superficie

## Chargement du data frame après toutes les étapes de data management réalisé

In [75]:
#A mettre en csv
df_filtre_final.to_csv('df_final.csv', index=False)

## Sélection de colonnes servant pour la data visualisation

In [77]:
list_drop = ["numero_disposition","adresse_numero", "adresse_suffixe", "adresse_nom_voie", "adresse_code_voie", "lot1_numero", "lot1_surface_carrez", "lot2_numero", 
            "lot2_surface_carrez", "lot3_numero", "lot3_surface_carrez" , "lot4_numero" , "lot4_surface_carrez" , "lot5_numero" , "lot5_surface_carrez", 
             "code_type_local", "code_nature_culture", "code_nature_culture_speciale", "nature_culture", "nature_culture_speciale", "code_postal_nb",
             "valeur_fonciere_par_id"]


clean_df_analyse = df_filtre_final.drop(columns=list_drop)

In [78]:
#Chargement  en csv
clean_df_analyse.to_csv('clean_df_analysis.csv', index=False)